In [7]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/skylerroh/mids/tldr-reddit-summarization')
from preprocess import utils
import json
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import inspect

In [5]:
tf.__version__

'2.1.0'

# Create TFRecords for Model Training and PEGASUS library

In [6]:
reddit_posts = utils.load_reddit_data_from_json()

0
1000000
2000000
3000000


In [36]:
subreddits, content, summary, titles, raw_content = utils.extract_tensors(
    [post for post in reddit_posts if 'title' in post and post.get('subreddit') in utils.top50subreddits])

In [37]:
features_dataset = tf.data.Dataset.from_tensor_slices({'topics':subreddits, 
                                                       'inputs': content, 
                                                       'targets': summary,
                                                       'titles': titles,
                                                       'raw_inputs': raw_content})

In [11]:
def write_tfrecord_file(dataset, filename):
    serialized_features_dataset = dataset.map(utils.tf_serialize_example)
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [13]:
# write_tfrecord_file(features_dataset.take(1000), 'reddit_small.tfrecord')
write_tfrecord_file(features_dataset, 'reddit_all.tfrecord')

## Read TFRecords

In [45]:
DATASET_SIZE = subreddits.shape[0]

train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

full_dataset = utils.build('reddit_all.tfrecord', True)
full_dataset = full_dataset.shuffle(seed=20200705, buffer_size=2**20)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

datasets = {'train': train_dataset, 'eval': val_dataset, 'test': test_dataset}

In [42]:
for key, dataset in datasets.items():
    filename = f"reddit_{key}.tfrecord"
    write_tfrecord_file(dataset, filename)

# Split into subreddits

In [48]:
def filter_subreddit(record, subreddit):
    return record['topics'] == subreddit

In [50]:
for key in ('test', 'eval', 'train'):
    dataset = utils.build(f'reddit_{key}.tfrecord', True)
    dataset.cache()
    for subreddit in utils.top50subreddits:
        sub_r = dataset.filter(lambda record: filter_subreddit(record, subreddit))
        filename = f'subreddits_data/subreddit_{subreddit}_{key}.tfrecord'
        writer = tf.data.experimental.TFRecordWriter(filename)
        write_tfrecord_file(sub_r, filename)

# make smaller example sets

In [55]:
for i, subreddit in enumerate(utils.top50subreddits):
    print(i, "--", subreddit)
    for key, n in [('train', 2000), ('eval', 100), ('test', 100)]:
        dataset = utils.build(f'/Users/skylerroh/mids/tldr-reddit-summarization/preprocess/subreddits_data/subreddit_{subreddit}_{key}.tfrecord', False)
        sub_r = dataset.take(n)
        filename = f'/Users/skylerroh/mids/tldr-reddit-summarization/subreddits_data/subreddit_{subreddit}_{key}_{n}.tfrecord'
        writer = tf.data.experimental.TFRecordWriter(filename)
        write_tfrecord_file(sub_r, filename)

0 -- AskReddit
1 -- relationships
2 -- leagueoflegends
3 -- tifu
4 -- relationship_advice
5 -- trees
6 -- gaming
7 -- atheism
8 -- AdviceAnimals
9 -- funny
10 -- politics
11 -- pics
12 -- sex
13 -- WTF
14 -- explainlikeimfive
15 -- todayilearned
16 -- Fitness
17 -- IAmA
18 -- worldnews
19 -- DotA2
20 -- TwoXChromosomes
21 -- videos
22 -- DestinyTheGame
23 -- reddit.com
24 -- offmychest
25 -- buildapc
26 -- AskMen
27 -- personalfinance
28 -- summonerschool
29 -- technology
30 -- wow
31 -- NoFap
32 -- starcraft
33 -- dating_advice
34 -- askscience
35 -- Games
36 -- news
37 -- talesfromtechsupport
38 -- depression
39 -- pcmasterrace
40 -- Guildwars2
41 -- magicTCG
42 -- loseit
43 -- GlobalOffensive
44 -- electronic_cigarette
45 -- movies
46 -- self
47 -- Advice
48 -- Drugs
49 -- teenagers


In [12]:
for key, n in [('train', 1000), ('eval', 100)]:
    datasets = []
    for i, subreddit in enumerate(utils.top50subreddits):
        dataset = utils.build(f'/Users/skylerroh/mids/tldr-reddit-summarization/preprocess/subreddits_data/subreddit_{subreddit}_{key}.tfrecord', False)
        sub_r = dataset.take(n)
        shuffled_data = sub_r.shuffle(n)
        datasets.append(shuffled_data)
    filename = f'/Users/skylerroh/mids/tldr-reddit-summarization/preprocess/{key}_top50_{n*50//1000}k.tfrecord'
    writer = tf.data.experimental.TFRecordWriter(filename)
    write_tfrecord_file(tf.data.experimental.sample_from_datasets(datasets), filename)

In [5]:
dataset=utils.build('/Users/skylerroh/mids/tldr-reddit-summarization/preprocess/train_top50_100k.tfrecord', False)
for record in dataset:
    x=1

# create extractive summary labels (greedy)

In [6]:
import preprocess.preprocessing

In [ ]:
for i, subreddit in enumerate(utils.top50subreddits):
    print(i, "--", subreddit)
    for key in ['train', 'eval', 'test']:
        dataset = utils.build(f'subreddits_data/subreddit_{subreddit}_{key}.tfrecord', False)
        sub_r = dataset.take(100)
        filename = f'subreddits_data/extractive_subreddit_{subreddit}_{key}_100.tfrecord'
        
        for example in sub_r:
            sent_list = greedy_selection(doc_sent_list, abstract_sent_list, summary_size_sentences, summary_size_unigrams):
                